In [1]:
import time
from datetime import datetime
import jsonlines
import json
from multiprocessing import Process, Queue
#from database.tweets_schema import insert_statuses, get_meta_id
#from utils.log import setup_logging
import tweepy
#from database.parser import parse_status
import gc
import os

In [2]:
import yaml

In [3]:
KEYS = yaml.load(open('../src/twitter-api-keys.yaml'), Loader=yaml.FullLoader)

In [4]:
consumer_key = KEYS.get('consumer_key')
consumer_secret = KEYS.get('consumer_secret')
access_token = KEYS.get('access_token')
access_token_secret = KEYS.get('access_token_secret')

In [5]:
#Twitter API auth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=30, retry_delay=30)

In [6]:
api.verify_credentials()

User(_api=<tweepy.api.API object at 0x0000017D6885CFC8>, _json={'id': 728337133824512001, 'id_str': '728337133824512001', 'name': 'Juan Acostupa', 'screen_name': 'jjacostupa', 'location': 'Lima, Peru', 'description': '👨\u200d💻📊🎯Ciencia de datos, la mayor parte del tiempo. Ingeniero mecatrónico en @UNIoficial.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 46, 'friends_count': 829, 'listed_count': 0, 'created_at': 'Thu May 05 21:34:39 +0000 2016', 'favourites_count': 745, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 319, 'lang': None, 'status': {'created_at': 'Thu Nov 12 02:24:04 +0000 2020', 'id': 1326712336590180352, 'id_str': '1326712336590180352', 'text': '@RoTorresT En ese top 20 hay más de 830 denuncias. En el gráfico anterior veo alrededor de 1000. \n\nAquí el principi… https://t.co/O7eSoabdNr', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'s

In [11]:
trends_available = api.trends_available()

In [46]:
trends_available[0]

{'name': 'Worldwide',
 'placeType': {'code': 19, 'name': 'Supername'},
 'url': 'http://where.yahooapis.com/v1/place/1',
 'parentid': 0,
 'country': '',
 'woeid': 1,
 'countryCode': None}

In [47]:
api.trends_place(23424765)

[{'trends': [{'name': 'Беларусь',
    'url': 'http://twitter.com/search?q=%D0%91%D0%B5%D0%BB%D0%B0%D1%80%D1%83%D1%81%D1%8C',
    'promoted_content': None,
    'query': '%D0%91%D0%B5%D0%BB%D0%B0%D1%80%D1%83%D1%81%D1%8C',
    'tweet_volume': None}],
  'as_of': '2020-11-12T06:21:22Z',
  'created_at': '2020-11-12T06:21:22Z',
  'locations': [{'name': 'Belarus', 'woeid': 23424765}]}]

In [16]:
api.trends_place(418440)

[{'trends': [{'name': 'Antero',
    'url': 'http://twitter.com/search?q=Antero',
    'promoted_content': None,
    'query': 'Antero',
    'tweet_volume': 13316},
   {'name': 'Premier',
    'url': 'http://twitter.com/search?q=Premier',
    'promoted_content': None,
    'query': 'Premier',
    'tweet_volume': 114625},
   {'name': '#QueNoJuegueLaSeleccion',
    'url': 'http://twitter.com/search?q=%23QueNoJuegueLaSeleccion',
    'promoted_content': None,
    'query': '%23QueNoJuegueLaSeleccion',
    'tweet_volume': None},
   {'name': '#MerinoNoNosRepresenta',
    'url': 'http://twitter.com/search?q=%23MerinoNoNosRepresenta',
    'promoted_content': None,
    'query': '%23MerinoNoNosRepresenta',
    'tweet_volume': 76244},
   {'name': '#SeriamejorpresidentequeMerino',
    'url': 'http://twitter.com/search?q=%23SeriamejorpresidentequeMerino',
    'promoted_content': None,
    'query': '%23SeriamejorpresidentequeMerino',
    'tweet_volume': None},
   {'name': '#ENHYPENHi_EP1',
    'url': 'htt

In [18]:
len(api.trends_place(418440)[0])

4

In [6]:
trends_peru = api.trends_place(23424919)

In [8]:
trends_peru[0].keys()

dict_keys(['trends', 'as_of', 'created_at', 'locations'])

In [8]:
aa = trends_peru[0]['trends']

In [12]:
aa[0]

{'name': '#MarchaNacional',
 'url': 'http://twitter.com/search?q=%23MarchaNacional',
 'promoted_content': None,
 'query': '%23MarchaNacional',
 'tweet_volume': 63893}

In [10]:
tweepy.__version__

'3.8.0'

In [18]:
os.makedirs('a/b/c')

In [20]:
bucket_name = 'dev-twitter-trends'
f's3://{bucket_name}/regions'

's3://dev-twitter-trends/regions'

In [22]:
os.path.isdir('../src')

True

In [43]:
list(enumerate([(x['name'], x['woeid'], x['country']) for x in trends_available]))

[(0, ('Worldwide', 1, '')),
 (1, ('Winnipeg', 2972, 'Canada')),
 (2, ('Ottawa', 3369, 'Canada')),
 (3, ('Quebec', 3444, 'Canada')),
 (4, ('Montreal', 3534, 'Canada')),
 (5, ('Toronto', 4118, 'Canada')),
 (6, ('Edmonton', 8676, 'Canada')),
 (7, ('Calgary', 8775, 'Canada')),
 (8, ('Vancouver', 9807, 'Canada')),
 (9, ('Birmingham', 12723, 'United Kingdom')),
 (10, ('Blackpool', 12903, 'United Kingdom')),
 (11, ('Bournemouth', 13383, 'United Kingdom')),
 (12, ('Brighton', 13911, 'United Kingdom')),
 (13, ('Bristol', 13963, 'United Kingdom')),
 (14, ('Cardiff', 15127, 'United Kingdom')),
 (15, ('Coventry', 17044, 'United Kingdom')),
 (16, ('Derby', 18114, 'United Kingdom')),
 (17, ('Edinburgh', 19344, 'United Kingdom')),
 (18, ('Glasgow', 21125, 'United Kingdom')),
 (19, ('Hull', 25211, 'United Kingdom')),
 (20, ('Leeds', 26042, 'United Kingdom')),
 (21, ('Leicester', 26062, 'United Kingdom')),
 (22, ('Liverpool', 26734, 'United Kingdom')),
 (23, ('Manchester', 28218, 'United Kingdom')),
 (

In [45]:
list(enumerate([(x['name'], x['woeid']) for x in trends_available if x['placeType']['name']=='Country']))

[(0, ('United Arab Emirates', 23424738)),
 (1, ('Algeria', 23424740)),
 (2, ('Argentina', 23424747)),
 (3, ('Australia', 23424748)),
 (4, ('Austria', 23424750)),
 (5, ('Bahrain', 23424753)),
 (6, ('Belgium', 23424757)),
 (7, ('Belarus', 23424765)),
 (8, ('Brazil', 23424768)),
 (9, ('Canada', 23424775)),
 (10, ('Chile', 23424782)),
 (11, ('Colombia', 23424787)),
 (12, ('Denmark', 23424796)),
 (13, ('Dominican Republic', 23424800)),
 (14, ('Ecuador', 23424801)),
 (15, ('Egypt', 23424802)),
 (16, ('Ireland', 23424803)),
 (17, ('France', 23424819)),
 (18, ('Ghana', 23424824)),
 (19, ('Germany', 23424829)),
 (20, ('Greece', 23424833)),
 (21, ('Guatemala', 23424834)),
 (22, ('Indonesia', 23424846)),
 (23, ('India', 23424848)),
 (24, ('Israel', 23424852)),
 (25, ('Italy', 23424853)),
 (26, ('Japan', 23424856)),
 (27, ('Jordan', 23424860)),
 (28, ('Kenya', 23424863)),
 (29, ('Korea', 23424868)),
 (30, ('Kuwait', 23424870)),
 (31, ('Lebanon', 23424873)),
 (32, ('Latvia', 23424874)),
 (33, ('Oma

In [41]:
local_trends = api.trends_place(23424936)

Rate limit reached. Sleeping for: 467


KeyboardInterrupt: 

In [38]:
local_trends[0]['trends']

[{'name': 'Dross',
  'url': 'http://twitter.com/search?q=Dross',
  'promoted_content': None,
  'query': 'Dross',
  'tweet_volume': 12433},
 {'name': '#YouTubeDOWN',
  'url': 'http://twitter.com/search?q=%23YouTubeDOWN',
  'promoted_content': None,
  'query': '%23YouTubeDOWN',
  'tweet_volume': 249026},
 {'name': 'Mario',
  'url': 'http://twitter.com/search?q=Mario',
  'promoted_content': None,
  'query': 'Mario',
  'tweet_volume': 89580},
 {'name': '#AquíPorTitanes',
  'url': 'http://twitter.com/search?q=%23Aqu%C3%ADPorTitanes',
  'promoted_content': None,
  'query': '%23Aqu%C3%ADPorTitanes',
  'tweet_volume': 11831},
 {'name': '#RPM40',
  'url': 'http://twitter.com/search?q=%23RPM40',
  'promoted_content': None,
  'query': '%23RPM40',
  'tweet_volume': None},
 {'name': '#CNCODejaVu',
  'url': 'http://twitter.com/search?q=%23CNCODejaVu',
  'promoted_content': None,
  'query': '%23CNCODejaVu',
  'tweet_volume': 82746},
 {'name': '#ConMatiNo',
  'url': 'http://twitter.com/search?q=%23Con

In [16]:
open('file:///C:/Users/USUARIO/Downloads/all-2020-11-13T00_40_48Z.json')

OSError: [Errno 22] Invalid argument: 'file:///C:/Users/USUARIO/Downloads/all-2020-11-13T00_40_48Z.json'

In [21]:
with open('C:/Users/USUARIO/Downloads/all-2020-11-13T01_07_14Z.json') as json_file:
    data = json.load(json_file)

In [32]:
import pandas as pd

df = pd.DataFrame(data)
df['query'] = df['query'].str.replace('%23','#')
df['query'] = df['query'].str.replace('%22','"')
df['query'] = df['query'].str.replace('+',' ')

In [41]:
df.query('name=="Peru"')

,name,url,promoted_content,query,tweet_volume,woeid,countrycode,as_of
1607,Peru,http://twitter.com/search?q=%23MarchaNacional,None,#MarchaNacional,71814.0,23424919,PE,2020-11-13T01:07:09Z
1608,Peru,http://twitter.com/search?q=%23VizcarraCorrupto,None,#VizcarraCorrupto,NaN,23424919,PE,2020-11-13T01:07:09Z
1609,Peru,http://twitter.com/search?q=%23MerinoNoNosRepr...,None,#MerinoNoNosRepresentas,NaN,23424919,PE,2020-11-13T01:07:09Z
1610,Peru,http://twitter.com/search?q=%22Richard+Swing%22,None,"""Richard Swing""",NaN,23424919,PE,2020-11-13T01:07:09Z
1611,Peru,http://twitter.com/search?q=Paraguay,None,Paraguay,45009.0,23424919,PE,2020-11-13T01:07:09Z
1612,Peru,http://twitter.com/search?q=%23MerinoPresident...,None,#MerinoPresidenteLegitimo,NaN,23424919,PE,2020-11-13T01:07:09Z
1613,Peru,http://twitter.com/search?q=%22MORAT+AL+AIRE%22,None,"""MORAT AL AIRE""",20519.0,23424919,PE,2020-11-13T01:07:09Z
1614,Peru,http://twitter.com/search?q=%23NADAxSamantha,None,#NADAxSamantha,20597.0,23424919,PE,2020-11-13T01:07:09Z
1615,Peru,http://twitter.com/search?q=movadef,None,movadef,NaN,23424919,PE,2020-11-13T01:07:09Z
1616,Peru,http://twitter.com/search?q=MVLL,None,MVLL,NaN,23424919,PE,2020-11-13T01:07:09Z


In [45]:
df['promoted_content']

0       None
1       None
2       None
3       None
4       None
        ... 
2516    None
2517    None
2518    None
2519    None
2520    None
Name: promoted_content, Length: 2521, dtype: object